In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# графики
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# для лемматизации
#import spacy
#sample_sentences = "Привет Миру! Как твои дела? Сегодня неплохая погода."
#nlp = spacy.load('ru2')
#nlp.add_pipe(nlp.create_pipe('sentencizer'), first=True)
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# импорт датасета
#df = pd.read_csv('C:\\Users\\vinov\\OneDrive\\Документы\\Универ\\Диплом\\Код\\archive\\lenta-ru-news.csv', low_memory = False, dtype = str)
df = pd.read_csv('C:\\Users\\vinov\\OneDrive\\Документы\\Универ\\Диплом\\Код\\archive\\labeled.csv', low_memory = False, dtype = str)
#print(df.topic.unique())
df.head()

# конвертировать в список
data = df.comment.values.tolist()
#data = df.text.values.tolist()

# удалить эл. адреса
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]

# удалить символы новой строки
data = [re.sub(r'\s+', ' ', sent) for sent in data]

# удалить кавычки
data = [re.sub(r'\"', '', sent) for sent in data]

pprint(data[:2])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:2])

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100) 
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatize(texts, allowed_postags = ['NOUN', 'ADJF', 'INFN', 'PRTF', 'GRND', 'ADV']):
    texts_out = []
    for sent in texts:
        butyavka = morph.parse(str(sent))[0]
        if butyavka.tag.POS in allowed_postags:
            texts_out.append(butyavka.lexeme)
    return texts_out

def lemmatize(texts, allowed_postags = ['NOUN', 'ADJF', 'INFN', 'PRTF', 'GRND', 'ADVB']):
    texts_out = []
    for sent in texts:
        to1 = []
        for part in sent:
            butyavka = morph.parse(part)[0]
            if butyavka.tag.POS in allowed_postags:
                to1.append(butyavka.normal_form)
        texts_out.append(to1)
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatize(data_words_bigrams)
print(data_lemmatized[:2])

['Верблюдов-то за что? Дебилы, бл... ',
 'Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. '
 'Если бы хохлов не было, кисель их бы придумал. ']
[['верблюдов', 'то', 'за', 'что', 'дебилы', 'бл'], ['хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'хохлов', 'еще', 'хуже', 'если', 'бы', 'хохлов', 'не', 'было', 'кисель', 'их', 'бы', 'придумал']]
[['верблюдов', 'дебил'], ['хохол', 'отдушина', 'затюканый', 'россиянин', 'вон', 'хохлов', 'хохлов', 'кисель']]
